# Complex-valued KAF applied on the Fashion-MNIST dataset

Demo from the following paper: https://arxiv.org/abs/1802.08026

## Step 1 - Install prerequisites

In [1]:
# The code requires autograd
!pip install autograd

  Running setup.py bdist_wheel for autograd ... - \ done
  Stored in directory: /content/.cache/pip/wheels/72/6f/c2/40f130cca2c91f31d354bf72de282922479c09ce0b7853c4c5
Successfully built autograd


In [2]:
# Download Fashion MNIST repository
!git clone https://github.com/zalandoresearch/fashion-mnist.git
  
# Add Fashion MNIST to path
import sys
sys.path.insert(0,'fashion-mnist')

Cloning into 'fashion-mnist'...
remote: Counting objects: 615, done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 615 (delta 5), reused 9 (delta 4), pack-reused 603
Receiving objects: 100% (615/615), 105.18 MiB | 22.20 MiB/s, done.
Resolving deltas: 100% (347/347), done.


## Step 2 - Data loading

In [5]:
from autograd import numpy as np
from sklearn import preprocessing, model_selection
from numpy.fft import fft2
import utils.mnist_reader as mnist_reader
      
# Load data
X_train, y_train = mnist_reader.load_mnist('fashion-mnist/data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('fashion-mnist/data/fashion', kind='t10k')
 
# Perform one-hot encoding on the output values  
onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
y_train = onehot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = onehot_encoder.fit_transform(y_test.reshape(-1, 1))
        
    
# Compute the Fourier transform (training)
Xfft_train = np.zeros((X_train.shape[0], X_train.shape[1])) + 1j*np.zeros((X_train.shape[0], X_train.shape[1]))
print('\nComputing Fourier transforms (training)... ')
for i in range(X_train.shape[0]):
  Xfft_train[i, :] = fft2(X_train[i, :].reshape(28, 28)).reshape(-1)
     
# Compute the Fourier transform (test)
Xfft_test = np.zeros((X_test.shape[0], X_test.shape[1])) + 1j*np.zeros((X_test.shape[0], X_test.shape[1]))
print('\nComputing Fourier transforms (test)... ')
for i in range(X_test.shape[0]):
  Xfft_test[i, :] = fft2(X_test[i, :].reshape(28, 28)).reshape(-1)
    
# Compute most significant coefficients
significance = np.mean(np.absolute(Xfft_train), axis=0)
significance_idx = np.argsort(significance)
    
# Remove non significant coefficients and scale the features   
scaler = preprocessing.MinMaxScaler(feature_range=(-1.0, +1.0))
Xfft_train = scaler.fit_transform(np.real(Xfft_train[:, significance_idx[-100:]])) + 1j*scaler.fit_transform(np.imag(Xfft_train[:, significance_idx[-100:]]))
Xfft_test = scaler.fit_transform(np.real(Xfft_test[:, significance_idx[-100:]])) + 1j*scaler.fit_transform(np.imag(Xfft_test[:, significance_idx[-100:]]))


Computing Fourier transforms (training)... 

Computing Fourier transforms (test)... 


## Step 3 - Define KAF neural network

In [0]:
# Auxiliary functions

def softmax(s):
    tmp = np.exp(s)
    return tmp/np.sum(tmp, axis=1).reshape(-1, 1)
    
def gauss_kernel(X, D, gamma=1.0):
    """
    Compute the 1D Gaussian kernel between all elements of a 
    NxH matrix and a fixed L-dimensional dictionary, resulting in a NxHxL matrix of kernel
    values.
    """
    return np.exp(- gamma*np.square(X.reshape(-1, X.shape[1], 1) - D))

In [0]:
###############################################################################
### SPLIT KAF NEURAL NETWORK ##################################################
###############################################################################

def init_split_kaf_nn(d, classes, layers=[], rs=np.random.RandomState(0), dict_size=20, boundary=2.0):
    """ 
    Initialize the parameters of a split KAF feedforward network.
    """
    
    # Add initial and final layer
    l = layers.copy()
    l.insert(0, d)
    l.append(classes)
    
    # Initialize dictionary
    D = np.linspace(-boundary, boundary, dict_size).reshape(1, 1, -1)
    
    # Rule of thumb for gamma
    interval = D[0,0,0]-D[0,0,1];
    sigma = 2*interval # empirically chosen
    gamma = 0.5/np.square(sigma)

    # Initialize parameters
    w = [(rs.randn(insize, outsize)*0.01 + 1j*rs.randn(insize, outsize)*0.01,   # weight matrix
                     rs.randn(outsize)*0.01 + 1j*rs.randn(outsize)*0.01,           # bias vector
                     rs.randn(1, outsize, dict_size)*0.5 + 0.0*1j, # KAF coefficients (real part)
                     rs.randn(1, outsize, dict_size)*0.5 + 0.0*1j) # KAF coefficients (imaginary part)
                     for insize, outsize in zip(l[:-1], l[1:])]
    
    # Remove the KAF from the last layer (softmax)
    w[-1] = (w[-1][0], w[-1][1])
    
    return w, (D, gamma)
        


def predict_split_kaf_nn(w, X, info):
    """
    Compute the outputs of a split KAF feedforward network.
    """
    
    (D, gamma) = info

    for info_layer in w:
        outputs = np.dot(X, info_layer[0]) + info_layer[1]
        
        if len(info_layer) > 2:
          # This is not the last layer
          K_real = gauss_kernel(np.real(outputs) + 0.0*1j, D, gamma)
          K_imaginary = gauss_kernel(np.imag(outputs) + 0.0*1j, D, gamma)
          X = np.sum(K_real*np.real(info_layer[2]), axis=2) + 1j*np.sum(K_imaginary*np.real(info_layer[3]), axis=2)
    
    return softmax(np.real(outputs)**2 + np.imag(outputs)**2)

## Step  4 - Training

In [0]:
# Utility class for iterating over the training data
class MiniBatchIterator():
    
    def __init__(self, X, y, B, random_seed=1):
        self.X = X
        self.y = y
        self.B = B
        self.idx = 0
        self.indices = np.arange(self.X.shape[0])
        np.random.seed(random_seed)

    # Create new mini-batch
    def next_minibatch(self, shuffle=True):
        
        if self.B == np.inf:
            return (self.X, self.y)
        
        # Shuffle the data if we are starting a new epoch
        if self.idx == 0 and shuffle:
            np.random.shuffle(self.indices)
            
        if self.idx + self.B >= self.X.shape[0]:
            end_idx = self.X.shape[0]
            reset = True
        else:
            end_idx = self.idx + self.B
            reset = False
        
        X_batch = self.X[self.indices[self.idx:end_idx]]
        y_batch = self.y[self.indices[self.idx:end_idx]]
        
        if reset:
            self.idx = 0
        else:
            self.idx += self.B
        
        return (X_batch, y_batch)

In [0]:
# Initialize network
w, info = init_split_kaf_nn(28 * 28, 10, [100, 100])

# Flatten the parameters
from autograd.misc.flatten import flatten
w_flat, unflattener = flatten(w)

In [0]:
# Define the cost function
def cross_entropy_loss(w_flat, x_batch, y_batch):
    """
    Mean cross-entropy over the batch of data.
    """
    pred = predict_split_kaf_nn(unflattener(w_flat), x_batch, info)
    return np.mean(-np.sum(y_batch*np.log(pred), axis=1))

# Define a function to compute the accuracy
def accuracy_score(y, y_pred):
    return np.mean(np.argmax(y, axis=1) == np.argmax(y_pred, axis=1))

In [0]:
# Optimization algorithm
def adagrad(f_grad, w, training_iterator, step_size = 0.01, fudge_factor = 1e-6, max_it=5000):
    # Adapted from:
    # https://github.com/benbo/adagrad

    gti = np.zeros(len(w)) + 1j*np.zeros(len(w))
    err = np.zeros(max_it)
    grad = np.zeros(max_it)

    for t in range(max_it):
        
        (X_batch, y_batch) = training_iterator.next_minibatch()
    
        err[t], g = f_grad(w, X_batch, y_batch)
        g = np.conj(g)
        grad[t] = np.sum(np.absolute(g)**2)  
        
        gti+=g**2
        adjusted_grad = g / (fudge_factor + np.sqrt(gti))
        w = w - step_size*adjusted_grad
        
        if t % 1000 == 0:
          print('Iteration {} ({:.2f} %), current loss is {}...'.format(t, t / max_it * 100, err[t]))
          
    return w, err, grad

In [0]:
# Main training part
from autograd import value_and_grad

# Set seed for PRNG
np.random.seed(0)

print('Training KAF networks...')

# Define the training iterator
batch_iterator = MiniBatchIterator(X_train, y_train, 40, random_seed=0)
        
# Define gradient
f_grad = value_and_grad(cross_entropy_loss)
            
# Train
w_final, loss_history, grad_history = adagrad(f_grad, w_flat, batch_iterator)
            
# Evaluate (needs a lot more iterations!)
print('Computing test error...')
y_pred = predict_split_kaf_nn(unflattener(w_final), X_test, info)
acc = accuracy_score(y_test, y_pred)
print('Final error on test set: ', acc)